<div class="markdown-google-sans">

## **Yolov5 Pedestrians and Crosswalks detection**
</div>
This notebook is used to detect pedestrians, crosswalks and whether it's safe for cars to cross the crosswalk using green bounding boxes or unsafe to cross using red bounding boxes.
Yolov5 weights is used for detection of pedestrians 
Training on custom dataset obtained using roboflow tool to detect crosswalks
Yolov5 custom weights (best.pt) weights is used for detection of crosswalks

---

TensorBoard tool is used for providing the measurements and visualizations needed during the training workflow for tracking experiment metrics like loss and accuracy and precision/recall graphs.


---


- 20 images and labels for training
- 9 images and labels for validation  
- 100 images for testing



<div class="markdown-google-sans">

## **Preparing the notebook**
</div>

*  mounting drive
*  clone yolov5

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
location = "/content/drive/MyDrive/crosswalk_" # --> location of test data

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies

import torch
import os

<div class="markdown-google-sans">

## Training crosswalk Data
</div>

Crosswalk folder:

*   train 
     1.   images
     2.   labels
*   valid
     1.   images
     2.   labels

*   data.yaml file

In [ ]:
!python train.py --img 416 --batch 16 --epochs 100 --data {location}/data.yaml --weights yolov5s.pt --cache

<div class="markdown-google-sans">

## Tensorboard
</div>

TensorBoard tool is used for providing the measurements and visualizations needed during the training workflow for tracking experiment metrics like loss and accuracy and precision/recall graphs.


In [ ]:
# Start tensorboard
# Launch after you have started training
# logs save in the folder "runs"
%load_ext tensorboard
%tensorboard --logdir runs

<div class="markdown-google-sans">

## Objection detection



In [ ]:
import glob

# Model
model = torch.hub.load('ultralytics/yolov5', 'yolov5s')  # or yolov5m, yolov5l, yolov5x, etc.
model2 = torch.hub.load('ultralytics/yolov5', 'custom', '/content/yolov5/runs/train/exp/weights/best.pt')  # custom trained model

model.conf = 0.5
model.classes = 0
images = '/content/drive/MyDrive/images/' # --> your test data
images = glob.glob(images + '/*.png')
boxes_pedestrian = []
boxes_crosswalk = []
for im in images:
  
  # Inference
  results1 = model(im)
  results2 = model2(im)

  boxes_pedestrian.append(list(results1.xyxy[0])) # ---> pedestrian bounding boxes for all images 
  boxes_crosswalk.append(list(results2.xyxy[0])) # ---> crosswalk bounding boxes for all images 


In [ ]:
import cv2 as cv
import numpy as np

%cd '/content/'
!mkdir output2
output = '/content/output2/' 
def dims(x):

    a = np.array([float(x[0]),float(x[1]),float(x[2]),float(x[3]),float(x[4])])
        
    x,y,w,h = a[0],a[1],a[2],a[3] 
    
    left = int(x) 
    top = int(y) 
    right = int(w)
    bottom = int(h) 
    conf = a[4]
    return left,top,right,bottom,conf

def draw_bounding_boxes(boxes_pedestrian,boxes_crosswalk,images):

  img_num = 0
  thickness = 3
  for box_pedestrian,box_crosswalk in zip(boxes_pedestrian,boxes_crosswalk):

    box_pedestrian = list(box_pedestrian)
    box_crosswalk = list(box_crosswalk)

    img_path = images[img_num]
    image = cv.imread(img_path)
    save_path = output
    dh, dw, _ = image.shape
    overlapped_c_p = []
    
    i = 0
    for p in box_pedestrian:
      while box_crosswalk:
        c = box_crosswalk[i]
        if ((min(p[2],c[2])>max(p[0],c[0])) & (min(p[3],c[3])>max(p[1],c[1]))):
          overlapped_c_p.append(np.array(c))
          box_crosswalk.remove(c)
          i = 0
        else:
          i = i+1
          
    for c in box_crosswalk:

      left_c,top_c,right_c,bottom_c,conf_c = dims(c)
      #green
      cv.rectangle(image, (left_c,top_c), (right_c, bottom_c ), (0, 128, 0), thickness)
      cv.putText(image, 'crosswalk safe '+ "%.3f" % conf_c, (left_c, top_c), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), thickness)
    for ov in overlapped_c_p:

      left_c,top_c,right_c,bottom_c,conf_c = dims(ov)
      #red
      cv.rectangle(image, (left_c,top_c), (right_c, bottom_c ), (0, 0, 255), thickness)
      cv.putText(image, 'crosswalk unsafe ' + "%.3f" % conf_c, (left_c, top_c), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), thickness)
    

    for p in box_pedestrian:
    
      left_p,top_p,right_p,bottom_p,conf_p = dims(p)
    
      cv.rectangle(image, (left_p,top_p), (right_p, bottom_p), (255, 0, 0), thickness)
      cv.putText(image, 'pedestrian'+ "%.3f" % conf_p, (left_p, top_p), cv.FONT_HERSHEY_SIMPLEX, 0.75, (255,255,255), thickness)
    
    cv.imwrite(os. path. join(output , "image_{}.jpg".format(img_num)), image)  # Save image localy 
    img_num+=1
        

In [ ]:
draw_bounding_boxes(boxes_pedestrian,boxes_crosswalk,images)